In [1]:
import os
import numpy as np
import pandas as pd
from tendrils import api, utils

from multiprocessing import Pool

utils.copy_from_other_config('config.ini')
OUT_DIR = 'data'
PROCESSES = 8

In [2]:
targets_list = api.get_targets()
targets_df = pd.DataFrame(targets_list)

flows_df = targets_df[targets_df.project=='flows']
flows_df = flows_df[flows_df.target_status=='target']
# remove targets without z
flows_df = flows_df[flows_df.redshift > 0]

In [3]:
def extract_target_phot(target_id):
    """Gets the light curves from ZTF (optical) and FLOWS
    website (NIR) for a given FLOWS target.
    
    Parameters
    ----------
    target_id: int
        Target's ID from FLOWS website.
        
    Returns
    -------
    opt_df: DataFrame
        DataFrame with information (mjd, mag, etc.) from the 
        optical light curves.
    nir_df: DataFrame
        DataFrame with information (mjd, mag, etc.) from the 
        NIR light curves.
    """
    # Opt
    taget_opt_phot = utils.download_ztf_photometry(target_id)
    opt_df = taget_opt_phot.to_pandas()
    opt_df.rename(columns={'time':'mjd'}, inplace=True) 

    # NIR
    taget_nir_phot = api.get_lightcurve(target_id)
    taget_nir_phot['mjd'] = taget_nir_phot['time'].value
    nir_df = taget_nir_phot.to_pandas()
    nir_df.rename(columns={'mag_raw':'mag', 
                           'mag_raw_error':'mag_err'}, 
                  inplace=True)
    
    return opt_df, nir_df

In [4]:
def pool_extract(target_name):
    """Creates a light-curve file with SNooPy format.
    
    target_name: str
        Target's name as given by FLOWS.
    """
    target_df = flows_df[flows_df.target_name==target_name]
    target_id = target_df.targetid.values[0]
    
    try:
        # photometry
        opt_df, nir_df = extract_target_phot(target_id)

        # metadata for SNooPy
        ra = target_df.ra.values[0]
        dec = target_df.decl.values[0]
        z = target_df.redshift.values[0]
        header = f'{target_name} {z} {ra} {dec}\n'

        outfile = os.path.join(OUT_DIR, f'{target_name}_snpy.txt')
        with open(outfile, 'w') as file:
            file.write(header)

            # NIR
            for filt in nir_df.photfilter.unique():
                if filt in ['J', 'H']:
                    # temporary patch
                    file.write(f'filter {filt}\n')
                    filt_df = nir_df[nir_df.photfilter==filt]

                    for mjd, mag, mag_err in zip(filt_df.mjd, 
                                                 filt_df.mag, 
                                                 filt_df.mag_err):
                        file.write(f' {mjd} {mag} {mag_err}\n')

            # Opt
            for filt in opt_df.photfilter.unique():
                file.write(f'filter {filt[0]}\n')
                filt_df = opt_df[opt_df.photfilter==filt]

                for mjd, mag, mag_err in zip(filt_df.mjd, 
                                             filt_df.mag, 
                                             filt_df.mag_err):
                    file.write(f' {mjd} {mag} {mag_err}\n')
    except Exception as message:
        print(f'{target_name} failed: {message}')

In [5]:
results = Pool(PROCESSES).map(pool_extract, (target_name for target_name \
                                             in flows_df.target_name))

2021fg failed: 'NoneType' object has no attribute 'to_pandas'
2021cp failed: 404 Client Error: Not Found for url: https://flows.phys.au.dk/api/lightcurve.php?target=415
2020uvd failed: 404 Client Error: Not Found for url: https://flows.phys.au.dk/api/lightcurve.php?target=29
2020aakx failed: 'NoneType' object has no attribute 'to_pandas'
2020zka failed: 404 Client Error: Not Found for url: https://flows.phys.au.dk/api/lightcurve.php?target=232
2020adle failed: 'NoneType' object has no attribute 'to_pandas'
2021zw failed: 'NoneType' object has no attribute 'to_pandas'
2021our failed: 'NoneType' object has no attribute 'to_pandas'
2021omp failed: 'NoneType' object has no attribute 'to_pandas'
2021bnd failed: 'NoneType' object has no attribute 'to_pandas'
2021pby failed: 'NoneType' object has no attribute 'to_pandas'
2021pfs failed: 'NoneType' object has no attribute 'to_pandas'
2021bsf failed: 'NoneType' object has no attribute 'to_pandas'
2021dch failed: 404 Client Error: Not Found for 

In [3]:
flows_df

,target_name,targetid,ra,decl,catalog_downloaded,discovery_mag,inserted,redshift,redshift_error,discovery_date,project,host_galaxy,ztf_id,target_status,sntype
3,2020oij,10,328.123800,42.906882,True,18.2600,2020-09-18 11:13:29.821990,0.021000,0.01,2020-07-09 11:41:19.000,flows,None,ZTF20abkdpsy,target,Ia
7,2020tug,18,359.866169,17.865214,True,20.6650,2020-10-19 09:56:19.613496,0.050000,NaN,2020-09-19 08:47:06.000,flows,SDSS J235927.40+175153.7,ZTF20acbovrt,target,Ia
11,2020uly,22,84.822415,63.641747,True,19.8397,2020-10-19 10:05:21.740483,0.036500,NaN,2020-09-28 08:54:42.998,flows,SDSS J053917.06+633832.0,ZTF20acfviac,target,Ia
12,2020xba,23,107.828742,44.379534,True,20.6400,2020-10-19 10:05:23.108215,0.040000,NaN,2020-10-13 11:42:36.288,flows,None,ZTF20acjhhqx,target,Ia
18,2020uvd,29,24.767741,75.323394,True,18.5400,2020-10-19 10:05:31.893755,0.036000,NaN,2020-10-04 10:06:14.400,flows,None,ZTF20acehyxd,target,Ia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2170,2022ctu,2181,149.177263,-19.940670,True,19.2469,2022-02-28 12:00:10.769229,0.058000,NaN,2022-02-19 08:09:52.001,flows,WISEA J095642.00-195630.1,ZTF22aabitze,target,Ia
2175,2022djw,2186,154.670226,10.779434,True,18.9900,2022-03-01 00:00:08.100446,0.051310,NaN,2022-02-26 10:47:21.120,flows,None,None,target,Ia
2182,2022djz,2193,192.165707,4.036617,True,19.2890,2022-03-02 06:00:08.173214,0.081000,NaN,2022-02-26 09:39:55.872,flows,None,ZTF22aabvelk,target,Ia
2185,2022csg,2196,181.499661,-16.892093,True,18.7700,2022-03-02 12:00:08.264988,0.076000,NaN,2022-02-17 08:33:57.024,flows,None,ZTF22aabiyjw,target,Ia
